### Get data from GP

In [1]:
import pandas as pd
import numpy as np

from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
data_gp = pd.read_csv('data/gp_training_data.csv')

In [ ]:
data_gp

In [ ]:
n_obj = len(data_gp['object_id'].unique())
n_obs = len(data_gp[(data_gp['object_id'] == 615) & (data_gp['passband'] == 0)])

In [ ]:
x_train = np.ndarray((n_obj,n_obs,6))

In [ ]:
unique_obj = data_gp['object_id'].unique()
y_train = []
for i, obj_id in enumerate(unique_obj):
    if i%1000 == 0: print(i)
    y_train.append(obj_id)
    for z in range(6):
        lc_temp = data_gp[(data_gp['object_id'] == obj_id) & (data_gp['passband'] == z)]
        x_train[i,:,z] = lc_temp['flux']
    
    
    

In [ ]:
x_train.nbytes/1e9

In [ ]:
np.save('train_format_gp.npy',x_train)

## Data preparation (after saving x_train)

In [2]:
import pandas as pd
import numpy as np

from tensorflow import keras
import matplotlib.pyplot as plt

In [3]:
meta_ = pd.read_csv('data/plasticc_train_metadata.csv.gz')
data_gp = pd.read_csv('data/gp_training_data.csv')

In [4]:
unique_obj = data_gp['object_id'].unique()

In [5]:
category_mapping = {90: 'SN1a', 67: 'SN1a-91bg', 52: 'SN1ax', 42: 'SN2', 62: 'SN1bc', 95: 'SLSN1', 15: 'TDE', 64: 'KN', 88: 'AGN', 92: 'RRL', 65: 'M-dwarf', 16: 'EB', 53: 'Mira', 6: 'Microlens'}

In [6]:
y_train = []
for i,obj_id in enumerate(unique_obj):
    class_ = meta_[meta_['object_id'] == obj_id]['target'].values[0]
    
    if category_mapping[class_] == 'AGN':
        y_train.append(1)
    else:
        y_train.append(0)
    
y_train = np.array(y_train)
np.save('data/train_format_gp_labels.npy',y_train)
y_train

array([0, 1, 0, ..., 0, 0, 0])

In [9]:
category_keys = [key for key in category_mapping.keys()]
num_classes = 14
y_train = []
for i, obj_id in enumerate(unique_obj):
    _class = meta_[meta_['object_id'] == obj_id]['target'].values[0]
    for c in range(num_classes):
        if category_keys[c] == _class:
            y_train.append(c)
            break
            
y_train = np.array(y_train)
np.save('data/train_format_gp_labels_all_categories.npy',y_train)
y_train.shape
y_train = keras.utils.to_categorical(y_train, num_classes)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

Simeon start from here with the Ferrari

In [ ]:
import pandas as pd
import numpy as np

from tensorflow import keras
import matplotlib.pyplot as plt

In [11]:
x_train = np.load('data/train_format_gp.npy')
y_train = np.load('data/train_format_gp_labels_all_categories.npy')
print(x_train.shape)
print(y_train.shape)


(7848, 1094, 6)
(7848,)


In [13]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

#num_classes = len(np.unique(y_train))
num_classes = 14
print(num_classes)

model = make_model(input_shape=x_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)
model.summary()

14
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1094, 6)]         0         
                                                                 
 conv1d_3 (Conv1D)           (None, 1094, 64)          1216      
                                                                 
 batch_normalization_3 (Batc  (None, 1094, 64)         256       
 hNormalization)                                                 
                                                                 
 re_lu_3 (ReLU)              (None, 1094, 64)          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 1094, 64)          12352     
                          

In [15]:
epochs = 100
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_all_classes.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/100
197/197 [==============================] - 7s 34ms/step - loss: 1.2458 - sparse_categorical_accuracy: 0.5938 - val_loss: 2.3905 - val_sparse_categorical_accuracy: 0.3713 - lr: 0.0010
Epoch 2/100
197/197 [==============================] - 6s 32ms/step - loss: 1.2359 - sparse_categorical_accuracy: 0.6050 - val_loss: 2.5729 - val_sparse_categorical_accuracy: 0.3701 - lr: 0.0010
Epoch 3/100
197/197 [==============================] - 6s 32ms/step - loss: 1.2127 - sparse_categorical_accuracy: 0.6072 - val_loss: 2.4428 - val_sparse_categorical_accuracy: 0.3803 - lr: 0.0010
Epoch 4/100
197/197 [==============================] - 6s 32ms/step - loss: 1.2437 - sparse_categorical_accuracy: 0.6058 - val_loss: 1.9399 - val_sparse_categorical_accuracy: 0.4108 - lr: 0.0010
Epoch 5/100
197/197 [==============================] - 6s 32ms/step - loss: 1.2106 - sparse_categorical_accuracy: 0.6126 - val_loss: 2.5587 - val_sparse_categorical_accuracy: 0.3739 - lr: 0.0010
Epoch 6/100
197/197 [====

197/197 [==============================] - 6s 32ms/step - loss: 1.0560 - sparse_categorical_accuracy: 0.6580 - val_loss: 2.7225 - val_sparse_categorical_accuracy: 0.4299 - lr: 5.0000e-04
Epoch 43/100
197/197 [==============================] - 6s 32ms/step - loss: 1.0332 - sparse_categorical_accuracy: 0.6676 - val_loss: 2.4752 - val_sparse_categorical_accuracy: 0.4338 - lr: 5.0000e-04
Epoch 44/100
197/197 [==============================] - 6s 32ms/step - loss: 1.0328 - sparse_categorical_accuracy: 0.6633 - val_loss: 2.9414 - val_sparse_categorical_accuracy: 0.4350 - lr: 5.0000e-04
Epoch 45/100
197/197 [==============================] - 6s 32ms/step - loss: 1.0130 - sparse_categorical_accuracy: 0.6704 - val_loss: 3.6386 - val_sparse_categorical_accuracy: 0.4051 - lr: 2.5000e-04
Epoch 46/100
197/197 [==============================] - 6s 32ms/step - loss: 1.0134 - sparse_categorical_accuracy: 0.6736 - val_loss: 3.6929 - val_sparse_categorical_accuracy: 0.4038 - lr: 2.5000e-04
Epoch 47/100


In [35]:
testdata = pd.read_csv("data/gp_test_data.csv")
unique_obj = testdata['object_id'].unique()
meta_ = meta_ = pd.read_csv('data/plasticc_test_metadata.csv')

category_keys = [key for key in category_mapping.keys()]
num_classes = 14
y_test = []
for i, obj_id in enumerate(unique_obj):
    _class = meta_.query("object_id == @obj_id")["true_target"].values[0]
    print(_class)
    for c in range(num_classes):
        if category_keys[c] == _class:
            y_test.append(c)
            break
            
y_test = np.array(y_test)
np.save('data/test_format_gp_labels_all_categories.npy',y_train)

42
42
42
90
90
90
42
42
90
90
42
42
42
90
42
90
90
42
42
42
16
90
90
42
42
90
90
42
42
67
90
62
42
90
62
90
90
90
42
42
42
42
90
90
90
42
42
90
90
42
42
42
90
42
42
42
42
42
90
42
42
42
42
90
42
42
42
62
90
90
42
90
90
90
90
42
90
42
42
42
42
42
42
90
42
42
42
42
90
90
42
42
90
42
90
90
42
90
90
42
42
90
90
90
993
90
42
42
42
42
90
90
42
42
90
92
42
42
42
90
90
90
42
42
42
52
42
90
90
52
90
90
42
90
42
90
42
90
42
42
90
90
42
90
62
90
52
42
42
42
90
42
42
42
90
90
42
90
90
90
90
90
90
90
90
42
42
90
90
90
52
90
90
90
90
42
90
90
42
90
62
90
42
42
42
42
42
67
42
90
90
90
90
42
67
42
42
42
90
42
42
90
90
42
42
42
42
42
42
42
42
42
67
42
42
90
42
90
90
42
52
90
90
42
42
42
90
52
42
42
42
88
90
90
90
42
42
90
42
62
65
88
42
42
42
90
42
90
90
42
42
42
42
42
42
90
42
62
42
90
42
42
42
42
42
42
88
90
90
42
42
42
42
42
42
42
67
90
42
92
90
42
90
62
90
90
42
42
42
42
62
62
90
90
90
42
90
90
90
90
42
42
42
42
42
90
90
42
42
62
42
42
42
42
42
90
90
42
42
42
42
42
90
42
42
42
62
42
42
42
90
42
42


90
90
90
90
90
90
42
42
90
90
90
90
42
42
90
90
42
90
42
90
42
90
42
90
90
90
90
90
42
42
90
42
90
90
42
42
42
42
62
42
42
90
42
67
88
42
90
42
90
42
42
90
42
90
42
42
42
92
62
42
42
42
62
42
42
90
42
62
62
42
42
42
42
42
90
42
90
90
90
90
90
42
42
90
42
42
42
42
42
90
88
52
42
62
42
42
42
42
90
90
90
42
42
42
90
42
90
90
90
90
42
42
42
90
42
90
90
42
52
42
90
42
90
992
90
90
90
90
90
42
42
42
42
42
42
42
42
90
90
42
42
90
90
42
42
67
42
90
90
42
90
90
42
90
90
992
90
42
42
90
90
42
90
42
88
42
90
42
62
42
42
42
62
95
90
90
90
90
90
42
42
62
90
90
42
90
42
67
90
90
90
42
90
90
90
90
42
42
42
90
90
90
90
42
42
42
90
62
42
42
42
42
42
42
90
90
90
52
42
42
42
42
62
90
90
42
42
90
42
42
90
90
42
62
42
42
90
42
42
88
90
90
42
90
90
90
42
90
42
90
90
993
42
90
42
42
90
90
90
42
90
42
62
42
90
90
42
90
42
62
90
42
90
62
90
90
52
42
42
42
42
90
42
62
42
42
62
42
90
90
90
42
62
42
90
67
90
42
42
90
90
42
42
42
90
88
42
42
42
42
42
42
90
42
42
90
42
42
42
90
90
62
42
42
90
42
90
65
993
90
42
90


In [36]:
model = keras.models.load_model("best_model_all_classes.h5")

x_test = np.load('data/test_format_gp.npy')
y_test = np.load('data/test_format_gp_labels_all_categories.npy')

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

ValueError: Data cardinality is ambiguous:
  x sizes: 5000
  y sizes: 0
Make sure all arrays contain the same number of samples.

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()